# Creating SQL data base using SQLLit

Simple Data warehousing done with SQLlit. It's not typically used for large-scale data warehousing like Snowflake, but it can be useful for small to medium-sized datasets, prototyping, or educational purposes, because  SQLite's nature as a lightweight, file-based database system.

### Step1 : Importing necessary library:

In [1]:
import sqlite3
import pandas as pd

### Step 2: Creating the 3 sample Dim tables and 1 Fact table to create star schema 

-Dim table 1 --> date table <br>
-Dim table 2 --> Product table<br>
-Dim table 3 --> Store table

-Fact table -->Sales_fact table

 IF NOt fuction is used to check the table already created.

Fcat table is the main table hold infromation about all the dim table. <br>
(Note: It's basic table formation can be modified based no needs by adding extra tables, assignee respective data types and filed names)

In [ ]:
def create_tables(conn):
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS date_dim (
        date_id INTEGER PRIMARY KEY,
        date TEXT,
        month TEXT,
        quarter TEXT,
        year INTEGER
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS product_dim (
        product_id INTEGER PRIMARY KEY,
        product_name TEXT,
        category TEXT,
        price REAL
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS store_dim (
        store_id INTEGER PRIMARY KEY,
        store_name TEXT,
        location TEXT,
        manager TEXT
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS sales_fact (
        sales_id INTEGER PRIMARY KEY,
        date_id INTEGER,
        product_id INTEGER,
        store_id INTEGER,
        quantity_sold INTEGER,
        sales_amount REAL,
        FOREIGN KEY (date_id) REFERENCES date_dim(date_id),
        FOREIGN KEY (product_id) REFERENCES product_dim(product_id),
        FOREIGN KEY (store_id) REFERENCES store_dim(store_id)
    );
    """)
    conn.commit()


### Step 3:  Data Ingestion <br>

Here we can ingest the data into table from any source by creating the connection 

In [145]:
def populate_tables(conn):
    cursor = conn.cursor()
    cursor.executemany("""
    INSERT OR IGNORE INTO date_dim (date_id, date, month, quarter, year) VALUES ;
    """, [
        (1, '2024-01-01', 'January', 'Q1', 2024),
        (2, '2024-01-02', 'January', 'Q1', 2024)
    ])
    cursor.executemany("""
    INSERT OR IGNORE INTO product_dim (product_id, product_name, category, price) VALUES ;
    """, [
        (1, 'Laptop', 'Electronics', 1200.00),
        (2, 'Smartphone', 'Electronics', 800.00)
    ])
    cursor.executemany("""
    INSERT OR IGNORE INTO store_dim (store_id, store_name, location, manager) VALUES ;
    """, [
        (1, 'Store A', 'New York', 'John Doe'),
        (2, 'Store B', 'San Francisco', 'Jane Smith')
    ])
    cursor.executemany("""
    INSERT OR IGNORE INTO sales_fact (sales_id, date_id, product_id, store_id, quantity_sold, sales_amount) VALUES ;
    """, [
        (1, 1, 1, 1, 2, 2400.00),
        (2, 2, 2, 2, 1, 800.00)
    ])
    conn.commit()


### Step4: Establishing the connection between the fact and Dim table to create star schema using SQL joint 

The results were established with pandas df. <br>
(Note: Even Snowflake schema can be create by creating more sub dim table )

In [11]:
def query_star_schema(conn):
    query = """
    SELECT 
        d.date,
        p.product_name,
        s.store_name,
        f.quantity_sold,
        f.sales_amount
    FROM 
        sales_fact f
    JOIN 
        date_dim d ON f.date_id = d.date_id
    JOIN 
        product_dim p ON f.product_id = p.product_id
    JOIN 
        store_dim s ON f.store_id = s.store_id
    WHERE 
        d.year = 2024;
    """
    df = pd.read_sql_query(query, conn)
    return df

### Once Query executed close the connection 

In [86]:
def main():
    # Path to your existing SQLite database file
    db_path = 'retail_sales.db'
    
    # Connect to the existing database
    conn = sqlite3.connect(db_path)
    
    # Query the star schema and get the results in a DataFrame
    df = query_star_schema(conn)
    
    # Print the results
    print(df)
    
    # Close the connection
    conn.close()

if __name__ == '__main__':
    main()

         date product_name store_name  quantity_sold  sales_amount
0  2024-01-01       Laptop    Store A              2        2400.0
1  2024-01-02   Smartphone    Store B              1         800.0


### New Query can be create with the existing database by establishing the new connection 

Below 4 new Query samples were executed.

In [121]:
def establish_connection(db_path):
    """Establish a connection to the SQLite database."""
    return sqlite3.connect(db_path)

def execute_query(conn, query):
    """Execute a given SQL query and return the results as a DataFrame."""
    return pd.read_sql_query(query, conn)

def run_queries(db_path):
    """Run multiple queries on the database and print the results."""
    # Establish connection
    conn = establish_connection(db_path)
    
    # Example query 1: Join all tables and retrieve sales data for the year 2024
    query1 = """
    SELECT 
        d.date,
        p.product_name,
        s.store_name,
        f.quantity_sold,
        f.sales_amount
    FROM 
        sales_fact f
    JOIN 
        date_dim d ON f.date_id = d.date_id
    JOIN 
        product_dim p ON f.product_id = p.product_id
    JOIN 
        store_dim s ON f.store_id = s.store_id
    WHERE 
        d.year = 2024;
    """
    df1 = execute_query(conn, query1)
    print("\nQuery 1 Results:")
    print(df1)
    
    # Example query 2: Total sales amount by product
    query2 = """
    SELECT 
        p.product_name,
        SUM(f.sales_amount) AS total_sales
    FROM 
        sales_fact f
    JOIN 
        product_dim p ON f.product_id = p.product_id
    GROUP BY 
        p.product_name;
    """
    df2 = execute_query(conn, query2)
    print("\nQuery 2 Results:")
    print(df2)
    
    # Example query 3: Total quantity sold by store
    query3 = """
    SELECT 
        s.store_name,
        SUM(f.quantity_sold) AS total_quantity
    FROM 
        sales_fact f
    JOIN 
        store_dim s ON f.store_id = s.store_id
    GROUP BY 
        s.store_name;
    """
    df3 = execute_query(conn, query3)
    print("\nQuery 3 Results:")
    print(df3)
    
    # Example query 4: Monthly sales
    query4 = """
    SELECT 
        d.month,
        SUM(f.sales_amount) AS total_sales
    FROM 
        sales_fact f
    JOIN 
        date_dim d ON f.date_id = d.date_id
    GROUP BY 
        d.month;
    """
    df4 = execute_query(conn, query4)
    print("\nQuery 4 Results:")
    print(df4)

    # Close the connection
    conn.close()

# Path to your existing SQLite database file
db_path = 'retail_sales.db'

# Run the queries
run_queries(db_path)


Query 1 Results:
         date product_name store_name  quantity_sold  sales_amount
0  2024-01-01       Laptop    Store A              2        2400.0
1  2024-01-02   Smartphone    Store B              1         800.0

Query 2 Results:
  product_name  total_sales
0       Laptop       2400.0
1   Smartphone        800.0

Query 3 Results:
  store_name  total_quantity
0    Store A               2
1    Store B               1

Query 4 Results:
     month  total_sales
0  January       3200.0


## one more query so we can do how many query performance we needed 
(Note: Each time once request done close the connection)

In [109]:
def run_new_query(db_path):
    """Run a new query on the database and print the results."""
    # Establish connection
    conn = establish_connection(db_path)
    
    # New query example: Sales by quarter
    query5 = """
    SELECT 
        d.quarter,
        SUM(f.sales_amount) AS total_sales
    FROM 
        sales_fact f
    JOIN 
        date_dim d ON f.date_id = d.date_id
    GROUP BY 
        d.quarter;
    """
    df5 = execute_query(conn, query5)
    print("Query 5 Results: Sales by Quarter")
    print(df5)
    
    # Close the connection
    conn.close()

# Run the new query
run_new_query(db_path)

Query 5 Results: Sales by Quarter
  quarter  total_sales
0      Q1       3200.0
